In [ ]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

#### Amostras por classe

In [ ]:
folder = Path("TCC_data")
folder_dataset = "labeled_dataset_typed"

In [ ]:
counter = dict()
for file in (folder/folder_dataset).glob("benchy_*"):
    df = pd.read_parquet(file)
    d = dict()
    d.update(df["class"].value_counts())
    for key,value in d.items():
        counter[key] = value + counter.get(key,0)

In [ ]:
convert_keys = {"temp_230":"Temp. 230", "temp_220":"Temp. 220", "healthy":"Saudável", "nozzle_03":"Nozzle 0,3","nozzle_02":"Nozzle 0,2", "loose_head":"Carro Folg."}

In [ ]:
conv_counter = {convert_keys[key]:value/1_000_000 for key,value in counter.items()}

In [ ]:
bars = plt.bar(conv_counter.keys(),conv_counter.values())

for bar in bars:
    height = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width() / 2, 
        height, 
        f'{height:.4}', 
        ha='center', 
        va='bottom',
        fontsize=12, 
        color='black'
    )
plt.xlabel("Classes")
plt.ylabel("Milhões de amostras")
plt.title("Quantidade de amostras por classe")
plt.show()
# plt.savefig(folder/folder_dataset/"amostras_classe.png")

#### Subamostragem

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters
sampling_rate = 500  # Sampling rate in Hz
duration = 0.1  # Duration in seconds
frequency = 0.1  # Frequency of the sine wave in Hz
num_samples = int(sampling_rate * duration)  # Total number of samples
highlight_interval = 16  # Interval of samples to highlight

# Time array
t = np.linspace(0, duration, num_samples, endpoint=False)

# Generate sine wave
sine_wave = np.sin(2 * np.pi * frequency * t)

# Generate white noise
noise = np.random.normal(0, 0.5, num_samples)

# Add noise to sine wave
signal = sine_wave + noise

# Extract highlighted samples
highlight_indices = np.arange(0, num_samples, highlight_interval)
highlighted_times = t[highlight_indices]
highlighted_signal = signal[highlight_indices]

# Calculate bar width
bar_width = duration / num_samples * 0.8  # Factor for slight spacing between bars

# Create a figure with two subplots
fig, axs = plt.subplots(1, 2, figsize=(16, 6))

# Plot the full signal with highlighted bars in red
for i in range(num_samples):
    color = 'darkred' if i in highlight_indices else 'navy'
    axs[0].bar(t[i], signal[i], color=color, width=bar_width)

axs[0].set_xlabel('Time (s)')
axs[0].set_ylabel('Amplitude')
axs[0].set_title('Full Signal with Highlighted Bars')
axs[0].grid(False)

# Capture the axis limits from the full signal plot
xlim = axs[0].get_xlim()
ylim = axs[0].get_ylim()

# Plot only the highlighted samples
axs[1].bar(highlighted_times, highlighted_signal, color='darkred', width=bar_width)
axs[1].set_xlabel('Time (s)')
axs[1].set_ylabel('Amplitude')
axs[1].set_title('Highlighted Samples')
axs[1].set_xlim(xlim)  # Apply the same x-axis limits
axs[1].set_ylim(ylim)  # Apply the same y-axis limits
axs[1].grid(False)

# Adjust layout to prevent overlap
plt.tight_layout()

# Show plot
plt.show()


#### FFT 2D IMAGE

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Assuming a generic image of size 44x4
image = np.random.rand(4, 1000)*16 - 8  # Generate a random 44x4 image
image[3,:] = image[0,:]

In [ ]:
# Perform 2D Discrete Fourier Transform (DFT)
dft_image = np.fft.fft2(image)

# Shift the zero-frequency component to the center of the spectrum
# dft_shifted = np.fft.fftshift(dft_image)

# Compute the magnitude spectrum (for visualization)
magnitude_spectrum = np.log10(np.abs(dft_image))

# Plot the original image and its magnitude spectrum
plt.figure(figsize=(12, 6))

# Original image
plt.subplot(1, 2, 1)
plt.imshow(image, cmap='gray')
plt.title('Original Image')
plt.colorbar()

# Magnitude Spectrum
plt.subplot(1, 2, 2)
plt.imshow(magnitude_spectrum, cmap='gray')  # Use log scale for better visualization
plt.title('Magnitude Spectrum')
plt.colorbar()

# Adjust layout and show plot
plt.tight_layout()
plt.show()


#### Filtro butterworth

In [ ]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt

# Filter specifications
pass_freq = 80      # Passband frequency (Hz)
stop_freq = 100      # Stopband frequency (Hz)
attenuation = 40    # Attenuation (dB)
sample_f = 3200     # Sampling frequency (Hz)

# Design the Butterworth filter
N, Wn = signal.buttord(wp=pass_freq, ws=stop_freq, gpass=1, gstop=attenuation, analog=False, fs=sample_f)
sos = signal.butter(N, Wn, fs=sample_f, btype="low", analog=False, output="sos")

# Compute frequency response
w, h = signal.sosfreqz(sos, worN=2000, fs=sample_f)

# Plot magnitude response
plt.figure(figsize=(6, 6))

# Magnitude Response
plt.subplot(2, 1, 1)
plt.plot(w[:201], 20 * np.log10(np.abs(h[:201])), 'b')
plt.title(f'Filtro Butterworth passa baixas de ordem {N}')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude (dB)')
plt.grid(True)
plt.axvline(pass_freq, color='r', linestyle='--', label='Frequência de corte')
plt.axvline(stop_freq, color='g', linestyle='--', label='Frequência de supressão')
plt.legend()

# Adjust layout
plt.tight_layout()
# Show plot
plt.show()